In [1]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/My Drive/dp_tomastik/code')
    !bash "/content/drive/My Drive/dp_tomastik/code/scripts/install_libs.sh"
    
import SimpleITK as sitk
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
import pandas as pd
import logging
import datetime
from torchio import RandomAffine, Compose, ZNormalization
from operator import itemgetter
from IPython.display import display, Markdown
from ipywidgets import widgets

from src.helpers import get_registration_transform_np, preview_3d_image, create_registration_list
from src.helpers import show_cuda_usage, preview_model_dataset_pred, preview_dataset
from src.helpers import get_threshold_info_df, get_rescaled_preds
from src.helpers import compare_prediction_with_ground_true, compare_one_prediction_with_ground_true
from src.helpers import get_img_outliers_pixels, get_raw_with_prediction
from src.helpers import get_rescaled_pred

from src.dataset import transform_input_with_registration, get_norm_transform
from src.dataset import get_full_res_cut, get_cut_lists, OARS_LABELS, get_dataset, get_dataset_info, get_dataset_transform
from src.dataset import split_dataset, copy_split_dataset

from src.model_and_training import prepare_model, train_loop, show_model_info, load_checkpoint_model_info
from src.model_and_training import iterate_model_v3v2
from src.model_and_training.getters.get_device import get_device
from src.model_and_training.architectures.unet_architecture_v3v2 import UNetV3v2

from src.consts import DATASET_MAX_BOUNDING_BOX, DESIRE_BOUNDING_BOX_SIZE

torch.manual_seed(20)
logging.basicConfig(filename='logs/model3v2_all_organs_jupyter.log', level=logging.DEBUG)

print('Dataset biggest bounding box wihtout spinal cord', DATASET_MAX_BOUNDING_BOX)
print('Cut target size', DESIRE_BOUNDING_BOX_SIZE)
print('Done Init')

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits

Dataset biggest bounding box wihtout spinal cord [56, 177, 156]
Cut target size [72, 192, 168]
Done Init


# Loading precourse neural network with datasets:
- loading fullres dataset (512x512)
- loading lowres dataset (32x32)
- loading precourse model
- parsing dataset to create cut dataset

In [2]:
datasets_params = ['train_dataset', 'valid_dataset', 'test_dataset']
filter_labels = OARS_LABELS.OARS_LABELS_LIST
if OARS_LABELS.SPINAL_CORD in filter_labels:
    filter_labels.remove(OARS_LABELS.SPINAL_CORD)

# low res
low_res_dataset = get_dataset(dataset_size=50, shrink_factor=16, filter_labels=filter_labels, unify_labels=True)
low_res_dataset.dilatate_labels(repeat=1)
low_res_dataset.to_numpy()
low_res_split_dataset_obj = split_dataset(low_res_dataset, train_size=40, valid_size=5, test_size=5)
train_low_res_dataset, valid_low_res_dataset, test_low_res_dataset = itemgetter(*datasets_params)(low_res_split_dataset_obj)

# full res
full_res_dataset = get_dataset(dataset_size=50, shrink_factor=1, filter_labels=filter_labels, unify_labels=False)
full_res_dataset.to_numpy()
full_res_split_dataset_obj = copy_split_dataset(full_res_dataset, low_res_split_dataset_obj)

# low res model - precourse model
epoch = 500
log_date = datetime.datetime(year=2020, month=10, day=27, hour=11, minute=45, second=30).strftime("%Y%m%d-%H%M%S")
model_name = f'{log_date}_3d_unet_PRECOURSE'

low_res_model_info = load_checkpoint_model_info(model_name, epoch, train_low_res_dataset, valid_low_res_dataset, test_low_res_dataset)
show_model_info(low_res_model_info)

# moving low res to gpu
low_res_model_info['device'] = get_device()
# low_res_model_info['device'] = 'cpu'
low_res_model_info['model'] = low_res_model_info['model'].to(low_res_model_info['device'])
low_res_model_info['model'].eval()

# cut res
cut_full_res_dataset = full_res_dataset.copy(copy_lists=False)
cut_full_res_dataset = get_cut_lists(low_res_model_info['model'],
                                     low_res_model_info['device'],
                                     low_res_dataset, 
                                     full_res_dataset, 
                                     cut_full_res_dataset, 
                                     low_res_mask_threshold=0.5)
cut_full_res_dataset.set_output_label(None)
cut_split_dataset_obj = copy_split_dataset(cut_full_res_dataset, low_res_split_dataset_obj)
cut_train_dataset, cut_valid_dataset, cut_test_dataset = itemgetter(*datasets_params)(cut_split_dataset_obj)

# moving low res model to cpu
low_res_model_info['device'] = 'cpu'
low_res_model_info['model'] = low_res_model_info['model'].to(low_res_model_info['device'])

CUDA using 16x dataset
filtering labels
filtering labels done
dilatating 1x dataset
parsing dataset to numpy
numpy parsing done
CUDA using 1x dataset
filtering labels
filtering labels done
parsing dataset to numpy
numpy parsing done


/home/nikolas/fiit-dp-thesis-code/venv/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'src.losses.dice_loss.DiceLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Model number of params: 298881, trainable 298881
get_cut_lists: Cutting index 0
get_full_res_cut: Removing 10/1335 outlier pixels
get_final_bounding_box_slice: box delta [21 48 24]
get_full_res_cut: Does cut and original label contain the same amount of pixels? True 1223526 1223526
get_cut_lists: Cutting index 1
get_full_res_cut: Removing 0/1416 outlier pixels
get_final_bounding_box_slice: box delta [24 16  8]
get_full_res_cut: Does cut and original label contain the same amount of pixels? True 1326052 1326052
get_cut_lists: Cutting index 2
get_full_res_cut: Removing 0/1873 outlier pixels
get_final_bounding_box_slice: box delta [ 20   0 -24]
get_full_res_cut: Does cut and original label contain the same amount of pixels? True 1890464 1890464
get_cut_lists: Cutting index 3
get_full_res_cut: Removing 0/1545 outlier pixels
get_final_bounding_box_slice: box delta [17 32  8]
get_full_res_cut: Does cut and original label contain the same amount of pixels? True 1560217 1560217
get_cut_lists: 

In [3]:
get_dataset_info(low_res_dataset, low_res_split_dataset_obj)

train 40, valid_size 5, test 5, full 50
train indices [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18, 20, 21, 22, 23, 24, 28, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
valid indices [6, 13, 19, 25, 38]
test indices [16, 26, 27, 29, 39]


In [4]:
preview_dataset(cut_full_res_dataset)

data max 3071, min -1024
label max 22, min 0


Output()

## Adding registration to dataset

In [5]:
atlas_ri = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/RI.mhd'))
atlas_brainstem_map = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/brain_stem_map.mhd'))
atlas_left_parotid_map = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/left_parotid_map.mhd'))
atlas_right_parotid_map = sitk.GetArrayFromImage(sitk.ReadImage('./data/PDDCA-1.2-atlas/probabilistic_atlas/right_parotid_map.mhd'))

atlas_ri.shape, atlas_brainstem_map.shape, atlas_left_parotid_map.shape, atlas_right_parotid_map.shape

((136, 120, 219), (136, 120, 219), (136, 120, 219), (136, 120, 219))

In [6]:
atlas_input = (atlas_ri[70:], atlas_left_parotid_map[70:])

showing registration example

In [11]:
dataset_input = cut_full_res_dataset.get_raw_item_with_label_filter(0)
tmp = get_registration_transform_np(dataset_input, atlas_input, numberOfIterations=500, show=False, preview=True, figsize=(10, 10))

Output()

registering whole dataset to probabilistic atlas

In [15]:
registration_list = create_registration_list(cut_full_res_dataset, atlas_input)

Registration done for index: 0
Registration done for index: 1
Registration done for index: 2
Registration done for index: 3
Registration done for index: 4
Registration done for index: 5
Registration done for index: 6
Registration done for index: 7
Registration done for index: 8
Registration done for index: 9
Registration done for index: 10
Registration done for index: 11
Registration done for index: 12
Registration done for index: 13
Registration done for index: 14
Registration done for index: 15
Registration done for index: 16
Registration done for index: 17
Registration done for index: 18
Registration done for index: 19
Registration done for index: 20
Registration done for index: 21
Registration done for index: 22
Registration done for index: 23
Registration done for index: 24
Registration done for index: 25
Registration done for index: 26
Registration done for index: 27
Registration done for index: 28
Registration done for index: 29
Registration done for index: 30
Registration done 

In [16]:
cut_full_res_dataset.data_list = registration_list

In [17]:
preview_3d_image(registration_list[0][0], figsize=(10, 10))
preview_3d_image(registration_list[0][1], figsize=(10, 10))

preview_3d_image(cut_full_res_dataset.label_list[1][0] == OARS_LABELS.PAROTID_GLAND_R, figsize=(10, 10))

Output()

Output()

Output()

# Training all organs models

checking list used for training single models

training each model

In [18]:
filter_labels = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels['SPINAL_CORD']

tmp_list = list(filter_labels.items())
labels_list = [tmp_list[10]]
for OAR_KEY, OAR_VALUE in labels_list:
    print(f"{OAR_KEY}, {OAR_VALUE}")

PITUITARY, 11


In [19]:
TRAIN_MODELS = False
if TRAIN_MODELS:
    for OAR_KEY, OAR_VALUE in labels_list:
        cut_full_res_dataset.set_output_label(OAR_VALUE)
        log_date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        model_name = f'{log_date}_3d_unet_lowres_model3v2_{OAR_KEY}'

        print(f'Training model with dataset label \'{OAR_KEY}\', value \'{OAR_VALUE}\'')
        print(f'folder \'{model_name}\'')
        cut_model_info = prepare_model(epochs=75,
                                       learning_rate=3e-4,
                                       in_channels=8,
                                       input_data_channels=1,
                                       output_label_channels=1,
                                       dropout_rate=0.2,
                                       train_batch_size=1,
                                       model_name=model_name,
                                       train_dataset=cut_train_dataset, 
                                       valid_dataset=cut_valid_dataset, 
                                       test_dataset=cut_test_dataset,
                                       model_class=UNetV3v2)
        show_model_info(cut_model_info)
        print('\n\n')
        train_loop(cut_model_info, iterate_model_fn=iterate_model_v3v2)
        print('\n\n')

        # clearing memory
        torch.cuda.empty_cache()

# Evaluating Model

In [20]:
filter_labels = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels['SPINAL_CORD']

tmp_list = list(filter_labels.items())
labels_list = [tmp_list[5], tmp_list[6], tmp_list[7], tmp_list[10], tmp_list[11], tmp_list[12]]
for OAR_KEY, OAR_VALUE in labels_list:
    print(f"{OAR_KEY}, {OAR_VALUE}")

OPT_NERVE_L, 6
OPT_NERVE_R, 7
OPT_CHIASMA, 8
PITUITARY, 11
PAROTID_GLAND_L, 12
PAROTID_GLAND_R, 13


In [21]:
def get_possible_models(oar_key):
    possible_models = [folder_name for folder_name in os.listdir('./models') if oar_key in folder_name]    
    
    return possible_models

### loading models to CPU 

In [22]:
models = dict()
for OAR_KEY, OAR_VALUE in labels_list:
    epoch = 75
    possible_models = get_possible_models(f"model3v2_{OAR_KEY}")
    if len(possible_models) <= 0:
        print(f'{OAR_KEY} Model: No avaiable model')
        continue

    model_name = possible_models[0]
    print(f'{OAR_KEY} Model: Loading model {model_name}')

    # loading model checkpoint
    cut_model_info = load_checkpoint_model_info(model_name, epoch, cut_train_dataset, cut_valid_dataset, cut_test_dataset, model_class=UNetV3v2)

    # moving model to cpu/cuda with eval mode
    cut_model_info['device'] = 'cpu'
    cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
    cut_model_info['model'].eval()
    cut_model_info['model'].disable_tensorboard_writing = True
    
    models[OAR_KEY] = cut_model_info

OPT_NERVE_L Model: No avaiable model
OPT_NERVE_R Model: No avaiable model
OPT_CHIASMA Model: No avaiable model
PITUITARY Model: Loading model 20210329-141131_3d_unet_lowres_model3v2_PITUITARY_atlas_gland_r
PAROTID_GLAND_L Model: No avaiable model
PAROTID_GLAND_R Model: Loading model 20210323-133350_3d_unet_lowres_model3v2_PAROTID_GLAND_R


## Testing Eval vs Train Mode

testing iteration function

In [23]:
cut_full_res_dataset.set_output_label(OARS_LABELS.PITUITARY)
cut_model_info = models['PITUITARY']
cut_model_info['device'] = get_device()
cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
cut_model_info['model'].disable_tensorboard_writing = True
    
model, model_name, optimizer, criterion = itemgetter('model', 'model_name', 'optimizer', 'criterion')(cut_model_info)
epochs, device, tensorboard_writer = itemgetter('epochs', 'device', 'tensorboard_writer')(cut_model_info)
train_dataloader, valid_dataloader, test_dataloader = itemgetter('train_dataloader',
                                                                 'valid_dataloader',
                                                                 'test_dataloader')(cut_model_info)
model.actual_epoch = 100

valid_loss, valid_dsc = iterate_model_v3v2(valid_dataloader, model, optimizer, criterion, device, is_eval=True)
print(valid_loss, valid_dsc)

cut_model_info['model'].disable_tensorboard_writing = True
cut_model_info['device'] = 'cpu'
cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])


Batch eval [1] loss 0.66669, dsc 0.33331
Batch eval [1] loss 0.43964, dsc 0.56036
Batch eval [1] loss 0.99367, dsc 0.00633
Batch eval [1] loss 0.29938, dsc 0.70062
Batch eval [1] loss 0.33634, dsc 0.66366
0.5471445083618164 tensor(0.4529, device='cuda:0')


In [27]:
SHOW_DSC_INFO = True
if SHOW_DSC_INFO:
    info_per_organs_df = {}
    models_info = list()
    for OAR_KEY, OAR_VALUE in labels_list:
        if OAR_KEY not in models:
            print(f'{OAR_KEY} Model: No avaiable model')
            continue

        # getting model to gpu
        cut_model_info = models[OAR_KEY]
        cut_model_info['device'] = get_device()
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
        cut_model_info['model'].eval()
        cut_model_info['model'].disable_tensorboard_writing = True

        # preparing dataset for comparison
        cut_full_res_dataset.set_output_label(OAR_VALUE)

        # calculating dsc predictions        
        info_df, preds, rescaled_preds = get_threshold_info_df(
                                    model=cut_model_info['model'], 
                                    dataset=cut_full_res_dataset, 
                                    device=cut_model_info['device'], 
                                    train_indices=cut_train_dataset.indices, 
                                    valid_indices=cut_valid_dataset.indices, 
                                    test_indices=cut_test_dataset.indices,
                                    step=0.5,
                                    transform_input_fn=transform_input_with_registration)
        info_per_organs_df[OAR_KEY] = info_df

        # moving model back to cpu
        cut_model_info['device'] = 'cpu'
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])

        # parsing data
        best_threshold_col = 'thres_rescaled_dsc_0.50'
        train_tmp_df = info_df[info_df['is_train']][best_threshold_col]
        valid_tmp_df = info_df[info_df['is_valid']][best_threshold_col]
        train_dsc = train_tmp_df.mean()
        valid_dsc = valid_tmp_df.mean()
        print(f'{OAR_KEY} Model: DSC train {round(train_dsc, 4)} valid {round(valid_dsc, 4)}')

        models_info.append({
            'oar_key': OAR_KEY,
            'model_name': model_name,
            # Train
            'train_dsc_mean': train_dsc,
            'train_dsc_std': train_tmp_df.std(),
            'train_dsc_median': train_tmp_df.median(),
            'train_dsc_min': train_tmp_df.min(),
            'train_dsc_max': train_tmp_df.max(),
            # Valid
            'valid_dsc_mean': valid_dsc,
            'valid_dsc_std': valid_tmp_df.std(),
            'valid_dsc_median': valid_tmp_df.median(),
            'valid_dsc_min': valid_tmp_df.min(),
            'valid_dsc_max': valid_tmp_df.max(),
            # Both
            'train_valid_mean_delta': train_dsc - valid_dsc
        })

    models_info_df = pd.DataFrame(models_info)
    
    tmp_df = models_info_df[['oar_key', 'train_dsc_mean', 'train_dsc_std', 'valid_dsc_mean', 'valid_dsc_std']].copy()
    tmp_df['train_dsc_mean'] = (tmp_df['train_dsc_mean'] * 100).round(2)
    tmp_df['valid_dsc_mean'] = (tmp_df['valid_dsc_mean'] * 100).round(2)
    tmp_df['train_dsc_std'] = (tmp_df['train_dsc_std'] * 100).round(2)
    tmp_df['valid_dsc_std'] = (tmp_df['valid_dsc_std'] * 100).round(2)
    
    display(tmp_df.mean().round(2))
    display(tmp_df.round(2))
    display(tmp_df.sort_values(by=['train_dsc_std']).round(2))
    display(models_info_df.sort_values(by=['train_dsc_mean']).drop(columns=['model_name']).round(2))
    display(models_info_df.sort_values(by=['train_valid_mean_delta']).drop(columns=['model_name']).round(2))

OPT_NERVE_L Model: No avaiable model
OPT_NERVE_R Model: No avaiable model
OPT_CHIASMA Model: No avaiable model
PITUITARY Model: DSC train 0.4059 valid 0.5102
PAROTID_GLAND_L Model: No avaiable model
PAROTID_GLAND_R Model: DSC train 0.8336 valid 0.8147


train_dsc_mean    61.98
train_dsc_std     19.09
valid_dsc_mean    66.24
valid_dsc_std     18.18
dtype: float64

,oar_key,train_dsc_mean,train_dsc_std,valid_dsc_mean,valid_dsc_std
0,PITUITARY,40.59,27.67,51.02,31.09
1,PAROTID_GLAND_R,83.36,10.51,81.47,5.26


,oar_key,train_dsc_mean,train_dsc_std,valid_dsc_mean,valid_dsc_std
1,PAROTID_GLAND_R,83.36,10.51,81.47,5.26
0,PITUITARY,40.59,27.67,51.02,31.09


,oar_key,train_dsc_mean,train_dsc_std,train_dsc_median,train_dsc_min,train_dsc_max,valid_dsc_mean,valid_dsc_std,valid_dsc_median,valid_dsc_min,valid_dsc_max,train_valid_mean_delta
0,PITUITARY,0.41,0.28,0.40,0.00,0.84,0.51,0.31,0.64,0.05,0.78,-0.10
1,PAROTID_GLAND_R,0.83,0.11,0.87,0.41,0.90,0.81,0.05,0.81,0.75,0.87,0.02


,oar_key,train_dsc_mean,train_dsc_std,train_dsc_median,train_dsc_min,train_dsc_max,valid_dsc_mean,valid_dsc_std,valid_dsc_median,valid_dsc_min,valid_dsc_max,train_valid_mean_delta
0,PITUITARY,0.41,0.28,0.40,0.00,0.84,0.51,0.31,0.64,0.05,0.78,-0.10
1,PAROTID_GLAND_R,0.83,0.11,0.87,0.41,0.90,0.81,0.05,0.81,0.75,0.87,0.02


In [25]:
if SHOW_DSC_INFO:
    tmp_column = 'is_train'
    
    try:
        print('OARS_LABELS.PAROTID_GLAND_R')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PAROTID_GLAND_R]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass

    try:   
        print('OARS_LABELS.PAROTID_GLAND_L')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PAROTID_GLAND_L]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass
        
    try: 
        print('OARS_LABELS.OPT_NERVE_L')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.OPT_NERVE_L]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass
        
    try: 
        print('OARS_LABELS.PITUITARY')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PITUITARY]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass

OARS_LABELS.PAROTID_GLAND_R


,dsc,rescaled_dsc,is_train,is_valid,is_test,thres_rescaled_dsc_0.00,thres_rescaled_dsc_0.50,thres_rescaled_dsc_1.00
index,,,,,,,,
31,0.837956,0.837956,True,False,False,0.004171,0.844588,2.060581e-10
47,0.843932,0.843932,True,False,False,0.004940,0.849554,1.738828e-10
30,0.842900,0.842900,True,False,False,0.003603,0.849899,2.386065e-10
3,0.845100,0.845100,True,False,False,0.004956,0.850423,1.733403e-10
10,0.854047,0.854047,True,False,False,0.009118,0.857408,9.402031e-11
7,0.860339,0.860339,True,False,False,0.004020,0.868321,2.138123e-10
48,0.864603,0.864603,True,False,False,0.003229,0.872472,2.662407e-10
24,0.869140,0.869140,True,False,False,0.006867,0.873347,1.249844e-10
14,0.871904,0.871904,True,False,False,0.007286,0.875796,1.177579e-10


OARS_LABELS.PAROTID_GLAND_L
OARS_LABELS.OPT_NERVE_L
OARS_LABELS.PITUITARY


,dsc,rescaled_dsc,is_train,is_valid,is_test,thres_rescaled_dsc_0.00,thres_rescaled_dsc_0.50,thres_rescaled_dsc_1.00
index,,,,,,,,
48,0.212520,0.212520,True,False,False,0.000011,0.203704,7.692307e-08
15,0.213073,0.213073,True,False,False,0.000026,0.229508,3.333333e-08
8,0.339329,0.339329,True,False,False,0.000047,0.343949,1.851852e-08
42,0.399898,0.404719,True,False,False,0.000026,0.390244,3.333333e-08
17,0.414386,0.414386,True,False,False,0.000071,0.419753,1.219512e-08
22,0.505022,0.505022,True,False,False,0.000024,0.518519,3.571428e-08
11,0.635498,0.635498,True,False,False,0.000047,0.628931,1.818182e-08
12,0.636411,0.636411,True,False,False,0.000046,0.662500,1.886792e-08
43,0.674048,0.674048,True,False,False,0.000146,0.679105,5.882353e-09


# Predictions merging and checking

In [ ]:
filter_labels_dict = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels_dict['SPINAL_CORD']

cut_full_res_dataset.set_output_label(filter_labels_dict)
preview_dataset(cut_full_res_dataset, preview_index=35)

In [ ]:
PARSE_CUT_DATASET = True
if PARSE_CUT_DATASET:
    from collections import defaultdict
    
    prediction_threshold = 0.5
    output_label_items = list(cut_full_res_dataset.output_label.items())[:]
    cut_dataset_predictions = defaultdict(lambda: defaultdict(lambda: np.zeros(cut_full_res_dataset[0][0][0].shape)))
    
    # for each label
    for label_index, val in enumerate(output_label_items[:]):
        OAR_KEY, OAR_VALUE = val
        # loading model
        if OAR_KEY not in models:
            print(f'{label_index+1}/{len(output_label_items)}: {OAR_KEY} Model: No avaiable model')
            continue
        print(f'{label_index+1}/{len(output_label_items)}: {OAR_KEY} Model: got model {datetime.datetime.now()}')

        # getting model to gpu
        cut_model_info = models[OAR_KEY]
        cut_model_info['device'] = get_device()
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
        cut_model_info['model'].eval()
        cut_model_info['model'].disable_tensorboard_writing = True

        # for label in whole dataset
        for index in range(len(cut_full_res_dataset)):
            prediction, rescaled_pred = get_rescaled_pred(cut_model_info['model'], cut_full_res_dataset, 
                                                          cut_model_info['device'], index, use_only_one_dimension=False)
    
            cut_dataset_predictions[index][OAR_VALUE] = prediction[0]
            # extended_cut_full_res_dataset.data_list[index][label_index + 1] = prediction
            # extended_cut_full_res_dataset.data_list[index][label_index + 1] = ((rescaled_pred > prediction_threshold) * 1).astype(np.int8)

        # moving model back to cpu
        cut_model_info['device'] = 'cpu'
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])

In [ ]:
if PARSE_CUT_DATASET:
    def custom_preview_dataset(dataset, predictions, preview_index=0, show_hist=False, use_transform=False):
        data, label = dataset.get_raw_item_with_label_filter(preview_index)  # equivalent dataset[preview_index]
        if use_transform:
            transform = get_dataset_transform()
            data, label = transform_input(data, label, transform)

        prediction = predictions[preview_index]
        max_channels = label.shape[0]
        max_slices = label.shape[1]

        print(f'data max {data.max()}, min {data.min()}')
        print(f'label max {label.max()}, min {label.min()}')

        def f(slice_index, label_channel):
            plt.figure(figsize=(20, 10))
            plt.subplot(1, 3, 1)
            plt.imshow(data[0, slice_index], cmap="gray")
            plt.subplot(1, 3, 2)
            plt.imshow(prediction[label_channel+1][slice_index])
            plt.subplot(1, 3, 3)
            plt.imshow(label[label_channel, slice_index])
            plt.show()

            if show_hist:
                plt.figure(figsize=(20, 10))
                plt.subplot(1, 2, 1)
                plt.hist(data.flatten(), 128)
                plt.subplot(1, 2, 2)
                plt.hist(label.flatten(), 128)
                plt.show()

        sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)
        labelChannelSlider = widgets.IntSlider(min=0, max=max_channels - 1, step=1, value=(max_channels - 1) / 2)
        ui = widgets.VBox([widgets.HBox([sliceSlider, labelChannelSlider])])
        out = widgets.interactive_output(f, {'slice_index': sliceSlider, 'label_channel': labelChannelSlider})
        # noinspection PyTypeChecker
        display(ui, out)
    
    index = cut_valid_dataset.indices[3]
    index = 35
    custom_preview_dataset(cut_full_res_dataset, cut_dataset_predictions, preview_index=index)

## Merging predictions

# TODO: refactor

MERGE_PREDICTIONS = False
if MERGE_PREDICTIONS:
    merged_predictions = [None] * len(extended_cut_full_res_dataset)
    for index in range(len(extended_cut_full_res_dataset)):
        # print(f"{index+1}/{len(extended_cut_full_res_dataset)}: Merging predictions to single label")
        data, label = extended_cut_full_res_dataset.get_raw_item_with_label_filter(index)

        new_data = np.zeros(data[0].shape, dtype=np.int16)
        for i in range(1, 22):
            new_data += data[i]

        merged_predictions[index] = new_data
    print('Merging done')

    # checking how many masks are overlapping
    for i, tmp_merged in enumerate(merged_predictions):
        display(f'scan id {i}: {np.where(tmp_merged == 1)[0].shape[0]}, {np.where(tmp_merged == 2)[0].shape[0]}, {np.where(tmp_merged == 3)[0].shape[0]}, {np.where(tmp_merged == 4)[0].shape[0]}')

# TODO: refactor

from src.losses import calc_dscm

def custom_preview_dataset2(dataset, preview_index=0, use_transform=False):
    data, label = dataset.get_raw_item_with_label_filter(preview_index)
    if use_transform:
        transform = get_dataset_transform()
        data, label = transform_input(data, label, transform)
        
    cut_data, cut_label = cut_full_res_dataset.get_raw_item_with_label_filter(preview_index)
    max_channels = label.shape[0]
    max_slices = label.shape[1]
    
    print(f'data max {data.max()}, min {data.min()}')
    print(f'label max {label.max()}, min {label.min()}')
    print(f'{data.shape}, {cut_data.shape}, {label.shape}, {cut_label.shape}')
    print(f'{data.dtype}, {cut_data.dtype}, {label.dtype}, {cut_label.dtype}')
    tmp_merged = merged_predictions[preview_index]
    print(f'{np.where(tmp_merged == 1)[0].shape},{np.where(tmp_merged == 2)[0].shape},{np.where(tmp_merged == 3)[0].shape},{np.where(tmp_merged == 4)[0].shape}')

    def f(slice_index, label_channel):
        print(f'{OARS_LABELS.OARS_LABELS_R_DICT[label_channel+1]}')
        tmp_tensor_label = torch.tensor(label[label_channel])
        tmp_tensor_prediciton = torch.tensor(data[label_channel+1])
        tmp_dsc = calc_dsc(tmp_tensor_label, tmp_tensor_prediciton)
        print(f'dsc {tmp_dsc}')

        plt.figure(figsize=(30, 20))

        plt.subplot(2, 3, 1).title.set_text('data')
        plt.imshow(cut_data[0, slice_index], cmap="gray")
        plt.subplot(2, 3, 2).title.set_text('label')
        plt.imshow(label[label_channel, slice_index])
        plt.subplot(2, 3, 3).title.set_text('prediciton')
        plt.imshow(data[label_channel+1, slice_index])
        # print(data.shape, np.sum(data[label_channel+1]), np.unique(data[1])[-1])
        print(f'slices with values > 0', (np.where(data[label_channel+1] > 0))[0])
        
        plt.subplot(2, 3, 4).title.set_text('merged prediction labels')
        plt.imshow(tmp_merged[slice_index], vmin=0, vmax=np.unique(tmp_merged)[-1])
        plt.subplot(2, 3, 5).title.set_text('merged labels ')
        plt.imshow(np.sum(label, axis=0)[slice_index])

        plt.show()

    sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)
    labelChannelSlider = widgets.IntSlider(min=0, max=max_channels - 1, step=1, value=(max_channels - 1) / 2)
    ui = widgets.VBox([widgets.HBox([sliceSlider, labelChannelSlider])])
    out = widgets.interactive_output(f, {'slice_index': sliceSlider, 'label_channel': labelChannelSlider})
    # noinspection PyTypeChecker
    display(ui, out)

custom_preview_dataset2(extended_cut_full_res_dataset, preview_index=35, use_transform=True)